# Analysis of Used Cars on German eBay

In this project, I will work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website.

The dataset was originally scraped and uploaded to Kaggle by user orgesleka however the dataset is not available on Kaggle anymore, but you can find it [here](https://data.world/data-society/used-cars-data).

## Information About the Dataset
**dateCrawled** - When this ad was first crawled. All field-values are taken from this date.<br>
**name** - Name of the car.<br>
**seller** - Whether the seller is private or a dealer.<br>
**offerType** - The type of listing.<br>
**price** - The price on the ad to sell the car.<br>
**abtest** - Whether the listing is included in an A/B test.<br>
**vehicleType** - The vehicle Type.<br>
**yearOfRegistration** - The year in which the car was first registered.<br>
**gearbox** - The transmission type.<br>
**powerPS** - The power of the car in PS.<br>
**model** - The car model name.<br>
**kilometer** - How many kilometers the car has driven.<br>
**monthOfRegistration** - The month in which the car was first registered.<br>
**fuelType** - What type of fuel the car uses.<br>
**brand** - The brand of the car.<br>
**notRepairedDamage** - If the car has a damage which is not yet repaired.<br>
**dateCreated** - The date on which the eBay listing was created.<br>
**nrOfPictures** - The number of pictures in the ad.<br>
**postalCode** - The postal code for the location of the vehicle.<br>
**lastSeenOnline** - When the crawler saw this ad last online.<br>

## Import Libraries and Explore Data

In [1]:
# import libraries

import pandas as pd
import numpy as np

# read csv into dataframe

autos = pd.read_csv('autos.csv', encoding='Latin-1') #Latin-1 is parameter is mandatory

In [2]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 20 columns):
dateCrawled            371528 non-null object
name                   371528 non-null object
seller                 371528 non-null object
offerType              371528 non-null object
price                  371528 non-null int64
abtest                 371528 non-null object
vehicleType            333659 non-null object
yearOfRegistration     371528 non-null int64
gearbox                351319 non-null object
powerPS                371528 non-null int64
model                  351044 non-null object
kilometer              371528 non-null int64
monthOfRegistration    371528 non-null int64
fuelType               338142 non-null object
brand                  371528 non-null object
notRepairedDamage      299468 non-null object
dateCreated            371528 non-null object
nrOfPictures           371528 non-null int64
postalCode             371528 non-null int64
lastSeen              

As it can be seen from the info method's results vehicleType, gearbox, model, fuelType, notRepairedDamage columns have lots of null values. These values should be handled.


The columns are all in English yet the values are all German. We can translate the German words and replace them with English counterparts. Let's see how many German words we should translate before moving on:

## Translate German Words

#### Seller Column

In [4]:
autos['seller'].unique()

array(['privat', 'gewerblich'], dtype=object)

We have two different German words Privat and Gewerblich. Which are Private and Commercial respectively. Let's create a dictionary for them for later use.

In [5]:
seller_words = {'privat':'private', 'gewerblich':'commercial'}

#### Offer Type Column

In [6]:
autos['offerType'].unique()

array(['Angebot', 'Gesuch'], dtype=object)

We have two different German words Angebot and Gesuch. Which are Offer and Request respectively. Let's create a dictionary for them for later use.

In [7]:
offerType_words = {'Angebot':'Offer', 'Gesuch':'Request'}

#### Vehicle Type Column

In [8]:
autos['vehicleType'].unique()

array([nan, 'coupe', 'suv', 'kleinwagen', 'limousine', 'cabrio', 'bus',
       'kombi', 'andere'], dtype=object)

We have three different German words Kleinwagen, kombi and andere. Which are Small Car, Station Wagon and Other respectively. Let's create a dictionary for them for later use.

In [9]:
vehicleType_words = {'kleinwagen':'small car', 'kombi':'station wagon', 'andere':'other'}

#### Gearbox Column

In [10]:
autos['gearbox'].unique()

array(['manuell', 'automatik', nan], dtype=object)

We have two different German words Manuell and Automatik. Which are Manual and Automatic respectively. Let's create a dictionary for them for later use.

In [11]:
gearbox_words = {'manuell':'manual', 'automatik':'automatic'}

#### Fuel Type Column

In [12]:
autos['fuelType'].unique()

array(['benzin', 'diesel', nan, 'lpg', 'andere', 'hybrid', 'cng',
       'elektro'], dtype=object)

We have many different German words Benzin, Andere and Elektro. Which are Gasoline, Other and Electric respectively. Let's create a dictionary for them for later use.

In [13]:
fuelType_words = {'benzin':'gasoline', 'andere':'other', 'elektro':'electric'}

#### Not Repaired Damage Column

In [14]:
autos['notRepairedDamage'].unique()

array([nan, 'ja', 'nein'], dtype=object)

We have two different German words Ja and Nein. Which are Yes and No respectively. Let's create a dictionary for them for later use.

In [15]:
notRepairedDamage_words = {'ja':'yes', 'nein':'no'}

We have finished all the columns needed translating from German to English. But now we have to map them to our dataset so the values can be translated in the dataset as well.

In [17]:
# iterate over the columns of the dataset and map related dictionaries to the columns
for column in autos.columns:
    if column == 'seller':
        autos[column] = autos[column].map(seller_words)
    elif column == 'offerType':
        autos[column] = autos[column].map(offerType_words)
    elif column == 'vehicleType':
        autos[column] = autos[column].map(vehicleType_words)
    elif column == 'gearbox':
        autos[column] = autos[column].map(gearbox_words)
    elif column == 'fuelType':
        autos[column] = autos[column].map(fuelType_words)
    elif column == 'notRepairedDamage':
        autos[column] = autos[column].map(notRepairedDamage_words)
    

In [18]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,private,Offer,480,test,NaN,1993,manual,0,golf,150000,0,gasoline,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,private,Offer,18300,test,NaN,2011,manual,190,NaN,125000,5,NaN,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",private,Offer,9800,test,NaN,2004,automatic,163,grand,125000,8,NaN,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,private,Offer,1500,test,small car,2001,manual,75,golf,150000,6,gasoline,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,private,Offer,3600,test,small car,2008,manual,69,fabia,90000,7,NaN,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


As it can be seen from the result all of the German words are translated successfully to English in the dataset. We will work on this dataset from now on.



Column names should be more descriptive and all *CamelCase* namings should be converted to *SnakeCase* for convenience.

### Rename Columns

In [19]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'kilometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [20]:
autos.rename({'dateCrawled':'crawled_date', 'offerType':'offer_type', 'vehicleType':'vehicle_type',
             'yearOfRegistration':'registration_year', 'powerPS':'power_ps', 'monthOfRegistration':'registration_month',
             'fuelType':'fuel_type', 'notRepairedDamage':'unrepaired_damage', 'dateCreated':'ad_created',
             'nrOfPictures':'nr_of_pictures', 'postalCode':'postal_code', 'lastSeen':'last_seen'}, axis=1, inplace=True)

In [21]:
autos.head()

,crawled_date,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,Golf_3_1.6,private,Offer,480,test,NaN,1993,manual,0,golf,150000,0,gasoline,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,private,Offer,18300,test,NaN,2011,manual,190,NaN,125000,5,NaN,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",private,Offer,9800,test,NaN,2004,automatic,163,grand,125000,8,NaN,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,private,Offer,1500,test,small car,2001,manual,75,golf,150000,6,gasoline,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,private,Offer,3600,test,small car,2008,manual,69,fabia,90000,7,NaN,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Columns are renamed in a way that all of them are in SnakeCase. They are all more descriptive than their original names.

### Further Exploration

The columns will be analyzed to check if there exist values that look unrealistically high or low namely the outliers. The outlier rows shall be removed from the dataset.

In [22]:
autos.describe(include='all')

,crawled_date,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,371528,371528,371528,371528,3.715280e+05,371528,150944,371528.000000,351319,371528.000000,351044,371528.000000,371528.000000,224169,371528,299468,371528,371528.0,371528.00000,371528
unique,280500,233531,2,2,NaN,2,3,NaN,2,NaN,251,NaN,NaN,3,40,2,114,NaN,NaN,182806
top,2016-03-24 14:49:47,Ford_Fiesta,private,Offer,NaN,test,small car,NaN,manual,NaN,golf,NaN,NaN,gasoline,volkswagen,no,2016-04-03 00:00:00,NaN,NaN,2016-04-06 13:45:54
freq,7,657,371525,371516,NaN,192585,80023,NaN,274214,NaN,30070,NaN,NaN,223857,79640,263182,14450,NaN,NaN,17
mean,NaN,NaN,NaN,NaN,1.729514e+04,NaN,NaN,2004.577997,NaN,115.549477,NaN,125618.688228,5.734445,NaN,NaN,NaN,NaN,0.0,50820.66764,NaN
std,NaN,NaN,NaN,NaN,3.587954e+06,NaN,NaN,92.866598,NaN,192.139578,NaN,40112.337051,3.712412,NaN,NaN,NaN,NaN,0.0,25799.08247,NaN
min,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,1000.000000,NaN,0.000000,NaN,5000.000000,0.000000,NaN,NaN,NaN,NaN,0.0,1067.00000,NaN
25%,NaN,NaN,NaN,NaN,1.150000e+03,NaN,NaN,1999.000000,NaN,70.000000,NaN,125000.000000,3.000000,NaN,NaN,NaN,NaN,0.0,30459.00000,NaN
50%,NaN,NaN,NaN,NaN,2.950000e+03,NaN,NaN,2003.000000,NaN,105.000000,NaN,150000.000000,6.000000,NaN,NaN,NaN,NaN,0.0,49610.00000,NaN
75%,NaN,NaN,NaN,NaN,7.200000e+03,NaN,NaN,2008.000000,NaN,150.000000,NaN,150000.000000,9.000000,NaN,NaN,NaN,NaN,0.0,71546.00000,NaN


In [23]:
autos['seller'].value_counts()

private       371525
commercial         3
Name: seller, dtype: int64

In [24]:
autos['offer_type'].value_counts()

Offer      371516
Request        12
Name: offer_type, dtype: int64

After having a look at the describe results, any columns that have mostly one value are the candidates to be dropped. **seller** and **offer_type** columns contain 2 different values but one value is extremely dominant compared to the other.

It is also wise to check other numerical columns such as **price** and **kilometer** to detect unrealistic values.

### Price Column Cleaning

In [25]:
autos['price'].describe()

count    3.715280e+05
mean     1.729514e+04
std      3.587954e+06
min      0.000000e+00
25%      1.150000e+03
50%      2.950000e+03
75%      7.200000e+03
max      2.147484e+09
Name: price, dtype: float64

Results show that minimum value for price column is '0' which is not a realistic price for a car. Also the maximum price for a car does not seem so realistic either. For further exploration of this matter let's check the minimum and maximum price values.

In [26]:
autos['price'].value_counts().sort_index(ascending=False).head(20)

2147483647     1
99999999      15
99000000       1
74185296       1
32545461       1
27322222       1
14000500       1
12345678       9
11111111      10
10010011       1
10000000       8
9999999        3
3895000        1
3890000        1
2995000        1
2795000        1
1600000        2
1300000        1
1250000        2
1234566        1
Name: price, dtype: int64

Maximum value for price column does not seem realistic at all. There exists some other made up values such as '99999999' etc. I will remove those made up values and outliers from the dataset. I think it would make sense to set the upper boundary to the value of 3895000 since a car could be that expensive and it would make perfect sense.      

In [27]:
autos['price'].value_counts().sort_index(ascending=True).head(30)

0     10778
1      1189
2        12
3         8
4         1
5        26
7         3
8         9
9         8
10       84
11        5
12        8
13        7
14        5
15       27
16        2
17        5
18        3
19        3
20       51
21        1
24        1
25       33
26        1
27        1
29        2
30       55
32        1
33        1
35       18
Name: price, dtype: int64

There exist 10778 cars with a price tag of 0 euros which is not realistic at all either. There exist some other values that do not make perfect sense and they will also be removed however I will set the lower boundary to 50 euros for the price of a car.

In [28]:
autos = autos[autos['price'].between(50,3895000)]

In [29]:
autos.shape

(359047, 20)

There was a total of 371528 cars in the dataset before the operations I made. It can be seen from the above result that now there exist 359047 cars remaining in the dataset. We can explore more about the remaining values for **price** column.

In [30]:
autos['price'].describe()

count    3.590470e+05
mean     6.096274e+03
std      1.822452e+04
min      5.000000e+01
25%      1.250000e+03
50%      3.099000e+03
75%      7.499000e+03
max      3.895000e+06
Name: price, dtype: float64

### Kilometer Column Cleaning

I set the boundaries for the **price** column and cleaned it. Now it is time to clean **kilometer** column as well. Let's begin with exploring the column.

In [31]:
autos['kilometer'].describe()

count    359047.000000
mean     125763.925057
std       39672.043220
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: kilometer, dtype: float64

In [32]:
autos['kilometer'].value_counts().sort_index(ascending=False).head(20)

150000    232119
125000     37293
100000     15404
90000      12334
80000      10883
70000       9657
60000       8584
50000       7528
40000       6314
30000       5925
20000       5507
10000       1842
5000        5657
Name: kilometer, dtype: int64

The maximum value for kilometer column is 150,000 which is perfectly fine for a car to have that value. Thus I will not remove high values for this column. However it is wise to check for the lower limit as well.

In [33]:
autos['kilometer'].value_counts().sort_index(ascending=True).head(20)

5000        5657
10000       1842
20000       5507
30000       5925
40000       6314
50000       7528
60000       8584
70000       9657
80000      10883
90000      12334
100000     15404
125000     37293
150000    232119
Name: kilometer, dtype: int64

Again I do not need to remove any rows with a made up value because it seems that the values are perfectly fine and already cleaned.

### Handling of Date Type Columns

Let's now move o to the date columns and understand the date range the data covers.

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:

- `crawled_date`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website


Right now, the **date_crawled**, **last_seen**, and **ad_created** columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. The other two columns are represented as numeric values, so we can use methods like Series.describe() to understand the distribution without any extra data processing.

In [34]:
autos['crawled_date'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.025554
2016-03-06    0.014461
2016-03-07    0.035636
2016-03-08    0.033461
2016-03-09    0.034143
2016-03-10    0.032642
2016-03-11    0.032759
2016-03-12    0.036299
2016-03-13    0.015786
2016-03-14    0.036346
2016-03-15    0.033433
2016-03-16    0.030194
2016-03-17    0.031656
2016-03-18    0.013107
2016-03-19    0.035252
2016-03-20    0.036366
2016-03-21    0.035672
2016-03-22    0.032464
2016-03-23    0.031990
2016-03-24    0.029899
2016-03-25    0.032756
2016-03-26    0.031979
2016-03-27    0.030199
2016-03-28    0.035051
2016-03-29    0.034124
2016-03-30    0.033564
2016-03-31    0.031876
2016-04-01    0.034179
2016-04-02    0.035129
2016-04-03    0.038861
2016-04-04    0.037669
2016-04-05    0.012762
2016-04-06    0.003125
2016-04-07    0.001607
Name: crawled_date, dtype: float64

As it can be seen from the result, crawl dates range from 2016-03-05 up to 2016-04-07. Now let's check **last_seen** column.

In [35]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2016-03-05    0.001267
2016-03-06    0.004100
2016-03-07    0.005192
2016-03-08    0.007924
2016-03-09    0.009823
2016-03-10    0.011458
2016-03-11    0.012954
2016-03-12    0.023242
2016-03-13    0.008414
2016-03-14    0.012174
2016-03-15    0.016324
2016-03-16    0.016435
2016-03-17    0.028729
2016-03-18    0.006893
2016-03-19    0.016349
2016-03-20    0.019875
2016-03-21    0.020050
2016-03-22    0.020538
2016-03-23    0.018042
2016-03-24    0.019176
2016-03-25    0.018981
2016-03-26    0.015951
2016-03-27    0.016680
2016-03-28    0.022198
2016-03-29    0.023289
2016-03-30    0.023718
2016-03-31    0.024264
2016-04-01    0.023950
2016-04-02    0.024986
2016-04-03    0.025356
2016-04-04    0.025540
2016-04-05    0.126903
2016-04-06    0.218843
2016-04-07    0.130384
Name: last_seen, dtype: float64

As it can be seen from the result, last seen dates range from 2016-03-05 up to 2016-04-07 which are the same as crawl dates. Now let's check **ad_created** column.

In [36]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

2014-03-10    0.000003
2015-03-20    0.000003
2015-06-11    0.000003
2015-06-18    0.000003
2015-08-07    0.000003
                ...   
2016-04-03    0.039053
2016-04-04    0.037775
2016-04-05    0.011592
2016-04-06    0.003117
2016-04-07    0.001543
Name: ad_created, Length: 114, dtype: float64

ad_created column is different from two other columns. It ranges from 2014-03-10 up to 2016-04-07. Ending dates are all similar for these three columns. 


### Convert Dates to Uniform Numeric Data

We can convert dates to uniform numeric data for convenience and for further purposes. Let's convert them!

In [37]:
autos['crawled_date'] = autos['crawled_date'].str[:10].str.replace('-','').astype(int)
autos['ad_created'] = autos['ad_created'].str[:10].str.replace('-','').astype(int)
autos['last_seen'] = autos['last_seen'].str[:10].str.replace('-','').astype(int)
autos

,crawled_date,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,kilometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,20160324,Golf_3_1.6,private,Offer,480,test,NaN,1993,manual,0,golf,150000,0,gasoline,volkswagen,NaN,20160324,0,70435,20160407
1,20160324,A5_Sportback_2.7_Tdi,private,Offer,18300,test,NaN,2011,manual,190,NaN,125000,5,NaN,audi,yes,20160324,0,66954,20160407
2,20160314,"Jeep_Grand_Cherokee_""Overland""",private,Offer,9800,test,NaN,2004,automatic,163,grand,125000,8,NaN,jeep,NaN,20160314,0,90480,20160405
3,20160317,GOLF_4_1_4__3TÜRER,private,Offer,1500,test,small car,2001,manual,75,golf,150000,6,gasoline,volkswagen,no,20160317,0,91074,20160317
4,20160331,Skoda_Fabia_1.4_TDI_PD_Classic,private,Offer,3600,test,small car,2008,manual,69,fabia,90000,7,NaN,skoda,no,20160331,0,60437,20160406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,20160314,Suche_t4___vito_ab_6_sitze,private,Offer,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,20160314,0,39576,20160406
371524,20160305,Smart_smart_leistungssteigerung_100ps,private,Offer,1199,test,NaN,2000,automatic,101,fortwo,125000,3,gasoline,smart,no,20160305,0,26135,20160311
371525,20160319,Volkswagen_Multivan_T4_TDI_7DC_UY2,private,Offer,9200,test,NaN,1996,manual,102,transporter,150000,3,NaN,volkswagen,no,20160319,0,87439,20160407
371526,20160320,VW_Golf_Kombi_1_9l_TDI,private,Offer,3400,test,station wagon,2002,manual,100,golf,150000,6,NaN,volkswagen,NaN,20160320,0,40764,20160324


Now let's check the two other columns which are already in numerical format. First start with **registration_year**:


In [38]:
autos['registration_year'].describe()

count    359047.000000
mean       2004.443964
std          80.907182
min        1000.000000
25%        1999.000000
50%        2004.000000
75%        2008.000000
max        9999.000000
Name: registration_year, dtype: float64

Minimum value for registration date is 1000 and maximum value is 9999. Both values are impossible to be real. There is no way a car exists in 1000s and since time travelling machines are not invented yet it is also impossible to be 9999. Let's further investigate this matter.


Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

In [39]:
autos['registration_year'].value_counts().sort_index(ascending=True).head(20)

1000    26
1001     1
1039     1
1111     2
1234     4
1255     1
1300     2
1400     1
1500     3
1600     2
1602     1
1800     4
1910    42
1911     1
1923     3
1925     1
1927     2
1928     2
1929     7
1930     4
Name: registration_year, dtype: int64

The lowest acceptable value for registration_year could be 1910 since there are 42 cars already registered and a year earlier than that makes no proper sense. Now let's decide the highest acceptable registration year.

In [40]:
autos['registration_year'].value_counts().sort_index(ascending=False).head(40)

9999      19
9450       1
9000       4
8888       2
8500       1
8200       1
8000       1
7800       1
7777       1
7500       1
7100       1
7000       4
6200       1
6000       6
5911       2
5900       1
5600       1
5555       2
5300       1
5000      15
4800       1
4500       2
4100       1
4000       3
3800       1
3700       1
3200       1
3000       5
2900       1
2800       1
2500       3
2290       1
2222       1
2200       1
2066       1
2019      17
2018    3798
2017    9943
2016    9158
2015    2857
Name: registration_year, dtype: int64

As I have mentioned above, because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. So we have to remove years down until 2016.

In [41]:
autos = autos[autos['registration_year'].between(1910,2016)]

In [42]:
autos['registration_year'].describe()

count    345151.000000
mean       2002.915608
std           7.201893
min        1910.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        2016.000000
Name: registration_year, dtype: float64

In [43]:
autos.shape

(345151, 20)

In [44]:
autos['registration_year'].value_counts(normalize=True)

2000    0.066270
1999    0.063549
2005    0.062607
2006    0.057882
2001    0.057027
          ...   
1928    0.000006
1927    0.000006
1940    0.000006
1925    0.000003
1911    0.000003
Name: registration_year, Length: 94, dtype: float64

## Exploring Price by Brand

Now let's explore the dataset even further. We can find the top 25 brands and calculate the average price for them.

In [45]:
autos['brand'].value_counts()

volkswagen        73081
bmw               37886
opel              36700
mercedes_benz     33441
audi              30921
ford              23799
renault           16406
peugeot           10417
fiat               8874
seat               6448
skoda              5433
mazda              5317
smart              4949
citroen            4816
nissan             4698
toyota             4473
hyundai            3449
mini               3248
sonstige_autos     3196
volvo              3160
mitsubishi         2843
honda              2601
kia                2393
suzuki             2204
alfa_romeo         2170
porsche            2146
chevrolet          1739
chrysler           1331
dacia               863
jeep                758
land_rover          751
daihatsu            747
subaru              728
jaguar              597
saab                505
daewoo              502
trabant             471
lancia              446
rover               437
lada                207
Name: brand, dtype: int64

There are many brands in the dataset and we can calculate the average price for each one of them but in this notebook only **top 25** of the brands will be selected and their average prices will be calculated. 

In [46]:
unique_brands = autos['brand'].unique()[:25]

#empty dictionary to hold aggregate data
top25_brands_means = {}

#iterate over unique brands and assign means to the brands in dictionary
for brand in unique_brands:
    selected_rows = autos.loc[autos['brand'] == brand, 'price']
    mean = selected_rows.mean()
    top25_brands_means[brand] = mean

In [47]:
top25_brands_means

{'volkswagen': 5475.510365211204,
 'audi': 9202.938779470263,
 'jeep': 12562.236147757256,
 'skoda': 6536.516289342904,
 'bmw': 8620.892810008974,
 'peugeot': 3278.8871076125565,
 'ford': 3794.7072986259927,
 'mazda': 4088.61576076735,
 'nissan': 4724.920178799489,
 'renault': 2447.3925393148847,
 'mercedes_benz': 8643.26847283275,
 'seat': 4556.774038461538,
 'honda': 4021.018069973087,
 'fiat': 2903.3492224476,
 'opel': 3044.620735694823,
 'mini': 10096.115455665025,
 'smart': 3646.393008688624,
 'hyundai': 5579.95216004639,
 'alfa_romeo': 4325.348387096774,
 'subaru': 4422.26510989011,
 'volvo': 5256.635126582279,
 'mitsubishi': 3421.8431234611326,
 'kia': 5864.963226076055,
 'suzuki': 4501.779491833031,
 'lancia': 3326.7802690582957}

Great! Now we have the results. We can see the top 25 brands in the dataset and their average prices. The brand with the highest average price is **jeep** whilst the brand with the lowest average is **renault**. 

### Link Between Mileage and Price

We have found out the price means for our dataset however we can make our investigation go even further. Now let's find the correlation between mileage and price means for these top 25 car brands.

In [48]:
#empty dictionary to hold aggregate data
top25_brands_mileage_means = {}

#iterate over unique brands and assign means to the brands in dictionary
for brand in unique_brands:
    selected_rows = autos.loc[autos['brand'] == brand, 'kilometer']
    mean = selected_rows.mean()
    top25_brands_mileage_means[brand] = mean

In [49]:
top25_brands_mileage_means

{'volkswagen': 128465.46982115734,
 'audi': 129501.63319426926,
 'jeep': 120600.26385224274,
 'skoda': 113595.61936315111,
 'bmw': 132920.33996727024,
 'peugeot': 124518.09542094653,
 'ford': 123716.54271187865,
 'mazda': 125791.79988715441,
 'nissan': 119454.02298850575,
 'renault': 127897.41557966598,
 'mercedes_benz': 130688.07750964384,
 'seat': 120600.96153846153,
 'honda': 125815.07112648981,
 'fiat': 116685.82375478928,
 'opel': 128797.41144414169,
 'mini': 93283.5591133005,
 'smart': 99647.40351586179,
 'hyundai': 104067.845752392,
 'alfa_romeo': 129232.71889400922,
 'subaru': 126373.62637362638,
 'volvo': 138219.93670886077,
 'mitsubishi': 127045.37460429124,
 'kia': 109258.25323861263,
 'suzuki': 106690.10889292195,
 'lancia': 124103.13901345292}

We can see the average mileage for the top25 brands above. Nearly all of them are on 100k kilometers band. Let's now check if there exists a relation between prices average and mileage average.

In [50]:
# brand prices means cast to pandas series
bmp_series = pd.Series(top25_brands_means)

# brand mileage means cast to pandas series
bmm_series = pd.Series(top25_brands_mileage_means)

# put series into a pandas dataframe
price_mileage_table = pd.DataFrame(bmp_series, columns=['mean_price'])

price_mileage_table['mean_mileage'] = bmm_series

# display only 1 digit after decimal point
pd.options.display.float_format = '{:,.1f}'.format

# display price_mileage_table
price_mileage_table

autos['kilometer'].describe()

count   345,151.0
mean    125,550.8
std      39,735.9
min       5,000.0
25%     100,000.0
50%     150,000.0
75%     150,000.0
max     150,000.0
Name: kilometer, dtype: float64

There doesn't seem to be a correlation sadly. Almost all of them are on 100k kilometers band. As it can be seen jeep branded cars are on 120k kilometers average but still it is the most expensive brand amongst all of them.

### Find the most common brand/model combinations

Now let's find the most frequent models for each car brand in German eBay.

In [51]:
unique_brands

array(['volkswagen', 'audi', 'jeep', 'skoda', 'bmw', 'peugeot', 'ford',
       'mazda', 'nissan', 'renault', 'mercedes_benz', 'seat', 'honda',
       'fiat', 'opel', 'mini', 'smart', 'hyundai', 'alfa_romeo', 'subaru',
       'volvo', 'mitsubishi', 'kia', 'suzuki', 'lancia'], dtype=object)

In [52]:
# dictionary to hold brand:model pairs

most_common_models = {}

# iterate over all brands

for brand in unique_brands:
    selected_rows = autos[autos['brand'] == brand]
    most_common_model = selected_rows['model'].value_counts().index[0] # get the most frequent model
    most_common_models[brand] = most_common_model

most_common_models

{'volkswagen': 'golf',
 'audi': 'a4',
 'jeep': 'grand',
 'skoda': 'octavia',
 'bmw': '3er',
 'peugeot': '2_reihe',
 'ford': 'focus',
 'mazda': '6_reihe',
 'nissan': 'micra',
 'renault': 'twingo',
 'mercedes_benz': 'c_klasse',
 'seat': 'ibiza',
 'honda': 'civic',
 'fiat': 'punto',
 'opel': 'corsa',
 'mini': 'cooper',
 'smart': 'fortwo',
 'hyundai': 'i_reihe',
 'alfa_romeo': '156',
 'subaru': 'impreza',
 'volvo': 'v40',
 'mitsubishi': 'colt',
 'kia': 'andere',
 'suzuki': 'andere',
 'lancia': 'ypsilon'}

We have the results! For every brand in the dataset we have found the most common model. We could turn this dictionary to a dataframe as well but we are not going to do operations on this result so we are leaving the dictionary as it is.

### Comparison of the prices for damaged cars and their non-damaged counterparts

Now let's find out how much cheaper are damaged cars than their non-damaged counterparts. We are going to use the most common model for each brand and compare their prices. Damaged most common model versus non-damaged most common model.

In [53]:
# iterate over the keys in most common models dictionary and find damaged, non-damaged cars price means for each brand


non_damaged_list = []
damaged_list = []

for key in most_common_models.keys():
    non_damaged_cars = autos[(autos['brand'] == key) & (autos['model'] == most_common_models[key]) & (autos['unrepaired_damage'] == 'no')]
    non_damaged_cars_mean = non_damaged_cars['price'].mean()
    non_damaged_list.append(non_damaged_cars_mean)

    
    damaged_cars = autos[(autos['brand'] == key) & (autos['model'] == most_common_models[key]) & (autos['unrepaired_damage'] == 'yes')]
    damaged_cars_mean = damaged_cars['price'].mean()
    damaged_list.append(damaged_cars_mean)
   

In [54]:
# zip function is used to add lists as rows to the dataframe
df = pd.DataFrame(zip(non_damaged_list, damaged_list), columns=['non_damaged_mean', 'damaged_mean'], index=most_common_models.values()) 
df

,non_damaged_mean,damaged_mean
golf,"5,931.7","1,913.0"
a4,"7,848.2","2,884.2"
grand,"16,163.7","4,733.8"
octavia,"8,156.7","4,042.4"
3er,"6,934.1","2,341.4"
2_reihe,"3,648.2","1,584.9"
focus,"4,743.8","1,419.2"
6_reihe,"4,292.3","1,746.5"
micra,"2,204.9",938.9
twingo,"1,782.0",677.5


Here are the results! As it can be seen, for each brand's most common cars both damaged and non-damaged price means are compared. Without proper calculation it can be directly seen from the dataframe that damaged cars are **50%** cheaper than their non-damaged counterparts.

## Conclusions:

In this notebook we have analysed dataset of used cars on German eBay. We started with translating German keywords to their English counterparts. Then we proceed to rename the columns in a way that we can easily understand what they all represent. After further exploration we have noticed that *offer type* and *seller* columns almost include only one value thus they are candidate columns to be dropped. Then we have cleaned the *price* column along with *kilometer* column. Lastly we handled date type columns and converted date types to numeric data for further purposes.

According to our exploration on the dataset:
    1. Most listed brands are respectively Volkswagen, BMW and Opel.
    2. Most expensive brands are respectively Jeep, Mini and Audi.
    3. There exists no proper correlation between a car's mileage and its price. Cars with the highest mileage are naturally cheaper than their lesser used versions.
    4. Most listed models of the most listed brands are respectively Golf, 3er, Corsa.
    5. Damaged cars are 50% cheaper than their non-damaged counterparts. 